In [18]:
import os
import sys
os.chdir("/Users/DanPfeiffer/Documents/code/data-collector/src")
sys.path.append(os.getcwd())
import pandas as pd
import numpy as np
from feature_eng import matchup_creator, box_score_feature_adder
import importlib
from datetime import datetime
from preprocess import preprocess_tr_stats, preprocess_box_scores
import json
import plotly.express as px

importlib.reload(matchup_creator)
importlib.reload(preprocess_tr_stats)
importlib.reload(preprocess_box_scores)
importlib.reload(box_score_feature_adder)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# stat_df_raw = pd.read_excel("../data/raw/full_stats.xlsx")

In [3]:
stat_df = stat_df_raw.copy()

In [5]:
stat_df.shape

(20128, 1589)

In [6]:
stat_df = preprocess_tr_stats.preprocess(stat_df)

In [7]:
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20128 entries, 0 to 20127
Columns: 215 entries, date to penalties_opponent_penalty_rate
dtypes: datetime64[ns, UTC](1), float64(213), object(1)
memory usage: 33.0+ MB


In [8]:
box_df_raw = pd.read_csv("../data/raw/box_scores_df.csv")
box_df_raw.shape

(6265, 72)

In [9]:
box_df = box_df_raw.copy()

In [10]:
box_df = preprocess_box_scores.preprocess(box_df)

In [11]:
box_processed = box_df.copy()

In [12]:
bsfa = box_score_feature_adder.BoxScoreFeatureAdder(box_df=box_processed)

In [13]:
box_scores_df = bsfa.add_features(df=box_processed)

adding location info for venue
adding location info for home team
adding location info for away team
adding local time
adding travel distance
adding days rest
adding box totals
adding spread results
adding totals results


/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:111: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{prefix}{spread}_home_cover"] = np.where(
/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:111: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{prefix}{spread}_home_cover"] = np.where(
/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:111: PerformanceWarning: DataFrame is highly fragmented.  This

In [14]:
box_scores_df.shape

(4863, 286)

In [18]:
for col in box_scores_df.columns:
    print(col)

id
uid
date
attendance
time_valid
neutral_site
conference_competition
play_by_play_available
recent
start_date
notes_type
notes_headline
broadcast_market
broadcast_name
type_id
type_abbreviation
venue_id
venue_full_name
venue_address_city
venue_address_state
venue_indoor
status_clock
status_display_clock
status_period
status_type_id
status_type_name
status_type_state
status_type_completed
status_type_description
status_type_detail
status_type_short_detail
status_is_tbd_flex
format_regulation_periods
home_id
home_uid
home_location
home_name
home_abbreviation
home_display_name
home_short_display_name
home_color
home_alternate_color
home_is_active
home_venue_id
home_logo
home_score
home_winner
home_current_rank
home_linescores
home_records
away_id
away_uid
away_location
away_name
away_abbreviation
away_display_name
away_short_display_name
away_color
away_alternate_color
away_is_active
away_venue_id
away_logo
away_score
away_winner
away_current_rank
away_linescores
away_records
game_id
sea

In [15]:
post_processed_box_df= box_scores_df.copy().sort_values(by="date")

In [16]:
def generate_features(row, creator):
    game_id = row['id']
    home_short_display_name = row['home_short_display_name']
    away_short_display_name = row['away_short_display_name'] 
    date = row['date']
    features_df = creator.create_matchup(game_id=game_id, 
                                         home_team_box_short_display_name=home_short_display_name, 
                                         away_team_box_short_display_name=away_short_display_name, 
                                         date=date)
    return features_df.iloc[0]

In [ ]:
importlib.reload(matchup_creator)

for decay_factor in np.arange(start=1.0, stop=0.85, step=-0.01):
    agg_method = "exp_weighted_mean"
    games_to_sample = 16
    experiment_name = f"{games_to_sample}_games_{agg_method}_decay{decay_factor}"
    print()
    print(experiment_name)
    print()
    mc = matchup_creator.MatchupCreator(stats_db=stat_df,
                                        games_to_sample=16, 
                                        aggregation_method=agg_method,
                                        decay_factor=decay_factor)
    features_df = post_processed_box_df.apply(lambda row: generate_features(row, mc), axis=1)
    df_final = pd.merge(post_processed_box_df, features_df, on='game_id', how='left')
    df_final.to_csv(f'../data/transformed/{experiment_name}_training.csv', index=False)

for games_to_sample in np.arange(start=4, stop=16, step=1):
    agg_method = "mean"
    experiment_name = f"{games_to_sample}_games_{agg_method}"
    print()
    print(experiment_name)
    print()
    mc = matchup_creator.MatchupCreator(stats_db=stat_df,
                                        games_to_sample=games_to_sample, 
                                        aggregation_method=agg_method,
                                        decay_factor=1.0)
    features_df = post_processed_box_df.apply(lambda row: generate_features(row, mc), axis=1)
    df_final = pd.merge(post_processed_box_df, features_df, on='game_id', how='left')
    df_final.to_csv(f'../data/transformed/{experiment_name}_training.csv', index=False)


16_games_exp_weighted_mean_decay1.0

creating matchups for 2005-01-02T18:00Z
creating matchups for 2005-01-02T21:05Z
creating matchups for 2005-01-02T21:15Z
creating matchups for 2005-01-03T01:30Z
creating matchups for 2005-01-08T21:30Z
creating matchups for 2005-01-09T01:00Z
creating matchups for 2005-01-09T18:00Z
creating matchups for 2005-01-09T21:30Z
creating matchups for 2005-01-15T21:30Z
creating matchups for 2005-01-16T01:00Z
creating matchups for 2005-01-16T18:00Z
creating matchups for 2005-01-16T21:30Z
creating matchups for 2005-01-23T20:00Z
creating matchups for 2005-01-23T23:30Z
creating matchups for 2005-02-06T23:30Z
creating matchups for 2005-09-09T01:00Z
creating matchups for 2005-09-11T17:00Z
creating matchups for 2005-09-11T20:15Z
creating matchups for 2005-09-12T00:30Z
creating matchups for 2005-09-13T01:00Z
creating matchups for 2005-09-18T17:00Z
creating matchups for 2005-09-18T20:05Z
creating matchups for 2005-09-18T20:15Z
creating matchups for 2005-09-19T00:30Z
cr

In [25]:
df_final.shape

(4863, 884)